In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
from boolrank import DualSiglip2Model
from my_processing import paths_to_dataset

# Load data
data_path = "training"
paths = [
    f"data/{data_path}.jsonl",
    "data/TAR_data.jsonl",
    "data/sysrev_conv.jsonl",
]
dataset = paths_to_dataset(
    paths,
    test_only_sources=['TAR', 'sysrev'],
    # train_sources=['pubmed-searchrefiner','pubmed-query','raw-jsonl']
    train_sources=['pubmed-searchrefiner','raw-jsonl']
)

data/training.jsonl
data/TAR_data.jsonl
data/sysrev_conv.jsonl


Finding similar: 100%|██████████| 3782/3782 [00:03<00:00, 1092.97it/s]


In [8]:
N = 10000
df = pd.DataFrame({
    "nl": dataset["train"]["nl_query"],
    "bool": dataset["train"]["bool_query"],
    "quality": dataset["train"]["quality"],
    "source": dataset["train"]["source"]
})
df = df.sample(min(N, df.shape[0])).reset_index(drop=True)
df

,nl,bool,quality,source
0,Clinical efficacy of home-use blue-light thera...,"((""Acne Vulgaris""[Mesh] OR Acne[tiab] OR Black...",0.003641,pubmed-searchrefiner
1,"[A prospective, randomized, open and comparati...","((""Acne Vulgaris""[Mesh] OR Acne[tiab] OR Black...",0.004232,pubmed-searchrefiner
2,"Environmental Needs, Barriers, and Facilitator...",1. (general surg* or colorectal surger* or cho...,0.417550,pubmed-searchrefiner
3,"A multicenter, randomized, split-face clinical...","((""Acne Vulgaris""[Mesh] OR Acne[tiab] OR Black...",0.014075,pubmed-searchrefiner
4,Differentiating psychogenic nonepileptic from ...,(Conversion Disorder*[tw] OR Functional Neurol...,0.417550,pubmed-searchrefiner
...,...,...,...,...
3777,Dopamine agonists for the treatment of cocaine...,((cocaine*[tiab]) AND (abuse*[tiab] OR addict...,0.549417,raw-jsonl
3778,Lifestyle advice to cancer survivors: a qualit...,perspectives OR views OR attitudes OR beliefs ...,0.125265,pubmed-searchrefiner
3779,Clonidine Increases the Likelihood That Abstin...,1. (opioid* or opiate* or analges* or pain kil...,0.156581,pubmed-searchrefiner
3780,Utility of (18) F-FDG PET/CT and CECT in conju...,((displaced[Title/Abstract] OR pancreas[Title/...,0.626325,pubmed-searchrefiner


In [9]:
model = DualSiglip2Model('BAAI/bge-small-en-v1.5', loss_type="clip")

embeddings = model.encode_text(df["nl"].tolist(), batch_size=1000).detach().cpu().numpy()

In [ ]:
tsne = TSNE(n_components=2,
            perplexity=30,
            learning_rate=200,
            n_iter=1000,
            random_state=42,
            init="pca")
coords = tsne.fit_transform(embeddings)

df["x"] = coords[:,0]
df["y"] = coords[:,1]

fig = px.scatter(
    df, x="x", y="y",
    color="source",               # color by data source
    # size="quality",               # optionally size by quality score
    hover_data={
        "nl": True,               # show the NL query on hover
        "bool": True,             # show the bool query
        "quality": True,
        "x": False, "y": False
    },
    title=f"t-SNE of {N} Text Embeddings"
)

fig.update_traces(marker=dict(opacity=0.7, line_width=0.5))
fig.update_layout(width=800, height=600)
fig.show()